# 2. Spark SQL, DataFrames and Datasets 

Spark SQL is a Spark module for structured data processing. 

__Features__

* More info about structure of data and computation performed
* Better optimization using extra information
* Same execution engine is used to compute a result, independent of which API/language used
* Able to switch between different APIs to express different transformations

## SQL

Able to execute SQL queries. Results will be returned as a _Dataset/DataFrame_. Can also interact with the SQL interface using the command-line

Note: Able to read data from Hive installations

## Datasets

Distributed collection of data. Available in Java and Scala, not Python or R (but both have benefits of Dataset API available) 

__Features__

* Strong typing and lambda functions
* Utilized SQL's optimized execution engine
* Manipulated using functional transformations (`map`, `flatMap`, `filter`)

## DataFrames

A Dataset organized into named columns. Similar to tables in relational databases or dataframes in Python, but with better optimization.

__Features__

* Constructed from multiple sources: structure data files, tables in Hive, external databases, ...
* DataFrame represented by a Dataset of `Rows`

# 2a. Getting Started

## SparkSession

Entry point to all functionality in Spark is `SparkSession` class

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
	.builder \
	.appName("Python Spark SQL basic example") \
	.config("spark.some.config.option", "some-value") \
	.getOrCreate()

## Creating DataFrames

Applicatons can create DataFrames from multiple data sources. 

In [2]:
# Create DataFrame from JSON file
df = spark.read.json('people.json')
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Untyped Dataset Operations (DataFrame Operations)

DataFrames are Dataset Rows in Scala and Java API. "untyped transformations" compared to "typed transformations".

Indexing is some with conventional pandas syntax `df['age']`

In [ ]:
# Print DataFrame format 
df.printSchema()

# Selecting columns
df.select("name").show()

# Applying some filters and functions
print("increment age by 1")
df.select(df['name'], df['age'] + 1).show()

print("select people older than 21")
df.filter(df['age'] > 21).show()

print("count people by age")
df.groupBy("age").count().show()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

increment age by 1
+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+

select people older than 21
+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

count people by age
+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



## Running SQL Queries Programmatically

`sql` function enables applications to run SQL queries and returns results as DataFrames

* `.sql(<query>)` -- create sql queries

In [ ]:
# Registered as SQL temporary view
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Global Temporary View

Session-scoped and will disappear if the session that created it terminates. A global temporary view allows a temporary view that is shared amongst all sessions and kept alive until the Spark application terminates.

Global temporary views are stored in a system preserved database `global_temp` and must be called from there.

* `.createGlobalTempView()` -- create a view that will persist between sessions

In [4]:
# Register as global temporary view
df.createGlobalTempView("people")

spark.sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
# Create a new session to showcase
spark.newSession().sql("SELECT * FROM global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Creating Datasets

Datasets use a specialized Encoder to serialize the objects for processing or transmitting over the network. Encoder code is dynamically generated for Spark to perform many operations without deserializing the bytes

__ONLY IN JAVA AND SCALA__

## Interoperating with RDDs

2 different methods to convert RDDs into Datasets:

1. Uses reflection to infer the schema of an RDD that contains specific types of objects - good if you already know the schema
2. Through a programmatic interface that allows constructing a schema to be applied to the existing RDD - for unknown column types 

### Method 1: Reflection

Rows are constructed by passing a list of key/value paris as kwargs to the Row class. Keys define column name, types are inferred by sampling the dataset

In [6]:
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession \
	.builder \
	.appName("Python Spark SQL basic example") \
	.config("spark.some.config.option", "some-value") \
	.getOrCreate()

# represents connection to a Spark cluster
# sparkContext is a legacy version of SparkSession which is a unified API
sc = spark.sparkContext

# Load text file, convert each line into Row
lines = sc.textFile("people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

# Infer schema and set DataFrame as a table
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

# SQL can run over DataFrames
teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <= 19")

# Results of queries are DataFrame objects
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teenNames:
	print(name)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 1 times, most recent failure: Lost task 1.0 in stage 2.0 (TID 5) (NCS-5CD8391B41.ncs.corp.int-ads executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)


### Programmatically Specifying the Schema

When a dictionary of kwargs cannot be defined ahead of time (e.g structure of records are all encoded in strings, or text dataset where each user will define their columns differently), a `DataFrame` can be created programmatically:

1. Create an RDD of tuples or lists from the original RDD
2. Create the schema represented by a `StructType` matching the structure of the tuples or lists created previously
3. Apply the schema to the RDD via `createDataFrame` method

In [7]:
# Import data types
from pyspark.sql.types import StringType, StructType, StructField


sc = spark.sparkContext

# Load text file and convert each line to Row
lines = sc.textFile("people.txt")
parts = lines.map(lambda l: l.split(','))

# Convert each line into a tuple
people = parts.map(lambda p: (p[0], p[1].strip()))

# Schema encoded in a string
schemaString = "name age"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Apply schema to the RDD
schemaPeople = spark.createDataFrame(people, schema)
schemaPeople.createOrReplaceTempView("people")

# SQL queries over the DataFrame
results = spark.sql("SELECT name FROM people")
results.show()

Py4JJavaError: An error occurred while calling o98.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 6) (NCS-5CD8391B41.ncs.corp.int-ads executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.comput

## Scalar Functions

Functions that return a single value per row. Spark SQL supports both built-in and user-defined Scalar functions

## Aggregate Functions

Functions that return a single value on a group of rows. Supports both built-in and user-defined Aggregate functions.

E.g `count()`, `count_distinct()`, `min()`, `max()`